<a href="https://colab.research.google.com/github/DrJHSIM/DeepLearningArticles/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 구현 환경 : Colab Pro, A100 GPU
- [논문](https://arxiv.org/pdf/1409.1556v6)의 Table.1에서 D열 VGG16 모델

In [22]:
!nvidia-smi


Sat May 11 07:31:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0              49W / 400W |  12491MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [14]:
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import Compose, ToTensor, Resize, CenterCrop, Normalize
from torch.utils.data import DataLoader
from torchvision import transforms

In [2]:
from torchsummary import summary

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device {device}")

Using device cuda


In [15]:
transform = Compose([
    Resize(226),
    CenterCrop(224),
    ToTensor(),
    Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
]
)

In [16]:
training_data = datasets.CIFAR10(
    root = 'data',
    train = True,
    download = True,
    transform = transform
)

test_data = datasets.CIFAR10(
    root = 'data',
    train = False,
    download = True,
    transform = transform
)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(test_data, batch_size = batch_size, shuffle = False)

In [18]:
class vgg16(nn.Module):
    def __init__(self):
        super().__init__()
        self.convetnets = nn.Sequential(
            #conv1
            nn.Conv2d(3, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2),
            #conv2
            nn.Conv2d(64, 128, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2),
            #conv3
            nn.Conv2d(128, 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2),
            #conv4
            nn.Conv2d(256, 512, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2),
            #conv5
            nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2),
        )
        self.classifier = nn.Sequential(
            #fc1
            nn.Linear(512*7*7 ,4096),
            nn.ReLU(),
            #fc2
            nn.Linear(4096, 4096),
            nn.ReLU(),
            #fc3
            nn.Linear(4096, 10)
        )

    def forward(self, x):
        x = self.convetnets(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model = vgg16().to(device)
print(model)

vgg16(
  (convetnets): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU()
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1),

In [8]:
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

In [20]:
def train_loop(dataloader, model, optimizer, loss_fn, batch_size):
    size = len(dataloader.dataset)
    model.train()
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        preds = model(x)
        loss = loss_fn(preds, y)

        #backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*batch_size
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    len_batch = len(dataloader)
    total_loss, correct = 0, 0
    model.eval()
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            preds = model(x)
            total_loss += loss_fn(preds, y).item()
            correct += (preds.argmax(1) == y).type(torch.float).sum().item()

    test_loss = total_loss/len_batch # average for batch_size
    test_correct = correct/len(dataloader.dataset)
    print(f"Test Error: \n Accuracy: {(100*test_correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [21]:
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n----------------------------")
    train_loop(train_dataloader, model, optimizer, loss_fn, batch_size)
    test_loop(test_dataloader, model, loss_fn)
print("Done")

Epoch 1
----------------------------
loss: 2.301365  [   64/50000]
loss: 2.073526  [ 6464/50000]
loss: 1.749542  [12864/50000]
loss: 1.754463  [19264/50000]
loss: 1.875210  [25664/50000]
loss: 1.927886  [32064/50000]
loss: 1.694125  [38464/50000]
loss: 1.555491  [44864/50000]
Test Error: 
 Accuracy: 43.3%, Avg loss: 1.529540 

Epoch 2
----------------------------
loss: 1.546105  [   64/50000]
loss: 1.429336  [ 6464/50000]
loss: 1.691718  [12864/50000]
loss: 1.241641  [19264/50000]
loss: 1.307496  [25664/50000]
loss: 1.373110  [32064/50000]
loss: 1.217593  [38464/50000]
loss: 1.195924  [44864/50000]
Test Error: 
 Accuracy: 59.2%, Avg loss: 1.149625 

Epoch 3
----------------------------
loss: 1.063754  [   64/50000]
loss: 1.201601  [ 6464/50000]
loss: 1.183943  [12864/50000]
loss: 1.120611  [19264/50000]
loss: 0.822925  [25664/50000]
loss: 0.951020  [32064/50000]
loss: 0.726578  [38464/50000]
loss: 1.001922  [44864/50000]
Test Error: 
 Accuracy: 62.2%, Avg loss: 1.075224 

Epoch 4
-----